In [1]:
import os
import torch
import monai
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd, Orientationd, ScaleIntensityRanged, CropForegroundd,
    RandCropByPosNegLabeld, RandFlipd, RandRotate90d, RandShiftIntensityd, EnsureTyped, DivisiblePadd
)
from monai.data import DataLoader, CacheDataset
from monai.networks.nets import SwinUNETR
from monai.utils import set_determinism
from monai.data.image_reader import NibabelReader
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch.nn.functional as F
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import re

In [2]:
 # Define directories
train_path = 'dataset/MICCAI_BraTS2020_TrainingData/'
# val_path = 'dataset/MICCAI_BraTS2020_ValidationData/'
modality_keys = ["flair"]
in_channels = len(modality_keys)
out_channels = len(modality_keys) 

In [3]:
def create_data_list(data_dir, patient_ids, lables, modality_keys):
    data_list = []
    for idx, patient in enumerate(patient_ids):
        patient_dir = f"{data_dir}{patient}/"
        if os.path.isdir(patient_dir):
            data_dict = {key: os.path.join(patient_dir, f"{patient}_{key}.nii") for key in modality_keys}
            data_dict['lable'] = lables[idx]
            data_list.append(data_dict)

    return data_list

In [4]:
def preprocess_labels(csv_file_path):
    df = pd.read_csv(csv_file_path)
    
    def extract_number(value):
        if isinstance(value, str):
            match = re.search(r'\d+', value)
            return int(match.group()) if match else None
        return value

    df['Survival_days'] = df['Survival_days'].apply(extract_number)
    df['Survival_days'] = pd.to_numeric(df['Survival_days'], errors='coerce')
    df = df.dropna(subset=['Survival_days'])
    df['Survival_days'] = df['Survival_days'].astype(int)
    
    return df['Survival_days'].values, df['Brats20ID'].values

In [5]:
all_labels, all_data_ids = preprocess_labels(f'{train_path}/survival_info.csv')
train_id, val_id, train_labels, val_labels = train_test_split(all_data_ids, all_labels, test_size=0.2, random_state=42)

train_data_list = create_data_list(train_path, train_id, train_labels, modality_keys)
val_data_list = create_data_list(train_path, val_id, val_labels, modality_keys)


# since we dont have lables for validation dataset not using it for validation
# val_data_list = create_data_list(val_path, modality_keys)
# val_labels = preprocess_labels(f'{val_path}/survival_evaluation.csv')



In [6]:
import nibabel as nib
import torch

# Load the NIfTI image
file_path = train_data_list[0]['flair']
img = nib.load(file_path)

# Get the image data as a numpy array
img_data = img.get_fdata()

# Convert the numpy array to a PyTorch tensor
img_tensor = torch.tensor(img_data, dtype=torch.float32)

# Print the shape of the tensor
print(f'Tensor shape: {img_tensor.shape}')


Tensor shape: torch.Size([240, 240, 155])


In [7]:
def get_transforms(modality_keys, pixdim=(2.0, 2.0, 2.0), spatial_size=(64, 64, 64)):
    transforms = Compose(
        [
            LoadImaged(keys=modality_keys, reader=NibabelReader()),
            EnsureChannelFirstd(keys=modality_keys),
            Spacingd(keys=modality_keys, pixdim=pixdim, mode=("bilinear")),
            Orientationd(keys=modality_keys, axcodes="RAS"),
            ScaleIntensityRanged(keys=modality_keys, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=modality_keys, source_key=modality_keys[0], allow_smaller=True),
            DivisiblePadd(keys=modality_keys, k=32),
            RandCropByPosNegLabeld(keys=modality_keys, label_key=modality_keys[0], spatial_size=spatial_size, pos=1, neg=1, num_samples=1, image_key=modality_keys[0], image_threshold=0),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=0),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=1),
            RandFlipd(keys=modality_keys, prob=0.5, spatial_axis=2),
            RandRotate90d(keys=modality_keys, prob=0.5, max_k=3),
            RandShiftIntensityd(keys=modality_keys, offsets=0.10, prob=0.5),
            EnsureTyped(keys=modality_keys),
        ]
    )
    return transforms

In [8]:
def get_val_transforms(modality_keys, pixdim=(2.0, 2.0, 2.0)):
    transforms = Compose(
        [
            LoadImaged(keys=modality_keys, reader=NibabelReader()),
            EnsureChannelFirstd(keys=modality_keys),
            Spacingd(keys=modality_keys, pixdim=pixdim, mode=("bilinear")),
            Orientationd(keys=modality_keys, axcodes="RAS"),
            ScaleIntensityRanged(keys=modality_keys, a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=modality_keys, source_key=modality_keys[0], allow_smaller=True),
            DivisiblePadd(keys=modality_keys, k=32),
            EnsureTyped(keys=modality_keys),
        ]
    )
    return transforms

In [9]:
train_transforms = get_transforms(modality_keys)
val_transforms = get_val_transforms(modality_keys)


In [10]:
train_ds = CacheDataset(
    data=train_data_list,
    transform=train_transforms,
    cache_rate=0.5,
    num_workers=4,
)
train_loader = DataLoader(train_ds,  batch_size=2, shuffle=True, num_workers=4)


Loading dataset:   0%|          | 0/94 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 94/94 [00:11<00:00,  7.92it/s]


In [11]:
val_ds = CacheDataset(
    data=val_data_list,
    transform=val_transforms,
    cache_rate=0.5,
    num_workers=4,
)
val_loader = DataLoader(val_ds, batch_size=2, shuffle=False, num_workers=2)

Loading dataset: 100%|██████████| 24/24 [00:03<00:00,  7.86it/s]


In [12]:
# Load the pretrained model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SwinUNETR(
    img_size=(64, 64, 64),
    in_channels=in_channels,
    out_channels=out_channels,
    feature_size=48,
    use_checkpoint=True,
).to(device)


# Load the saved weights
modality_used = "_".join(modality_keys)
model_save_path = f"model_saved/swin_unetr_{modality_used}_temp.pth"

model.load_state_dict(torch.load(model_save_path))
model.eval()

SwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(1, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0-1): 2 x SwinTransformerBlock(
            (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=48, out_features=144, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=48, out_features=48, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=48, out_features=192, bias=True)
              (linear2): Linear(in_feature

In [13]:
# # 2. Modify the SwinUNETR class to add a method for feature extraction
# class FeatureExtractorSwinUNETR(SwinUNETR):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
    
#     def extract_features(self, x):
#         print(" at start",x.size())
#         hidden_states = self.swinViT(x, self.normalize)
#         enc0 = self.encoder1(x)
#         print(" after enco 1",x.size())
#         enc1 = self.encoder2(hidden_states[0])
#         print(" after enco 2",enc1.size())
#         enc2 = self.encoder3(hidden_states[1])
#         print(" after enco 3",enc2.size())
#         enc3 = self.encoder4(hidden_states[2])
#         print(" after enco 4",enc3.size())
#         # dec3 = self.encoder10(hidden_states[3])
#         # print(" after dec3",dec3.size())
#         # return torch.cat([enc0, enc1, enc2, enc3, dec3], dim=1)
#         return torch.cat([enc0, enc1, enc2, enc3], dim=1)

In [14]:
class FeatureExtractorSwinUNETR(SwinUNETR):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def extract_features(self, x):
        hidden_states = self.swinViT(x, self.normalize)
        
        # Apply Global Average Pooling to each feature tensor
        pooled_features = [F.adaptive_avg_pool3d(state, (1, 1, 1)).squeeze(-1).squeeze(-1).squeeze(-1) 
                           for state in hidden_states]
        
        # Concatenate the pooled features
        concatenated_features = torch.cat(pooled_features, dim=1)
        
        print(f"Shape of concatenated features: {concatenated_features.shape}")
        
        return concatenated_features


In [15]:
# class FeatureExtractorSwinUNETR(SwinUNETR):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
    
#     def extract_features(self, x):
#         hidden_states = self.swinViT(x, self.normalize)
        
#         # Adaptive pooling to resize all hidden states to a common size
#         pooled_states = []
#         for state in hidden_states:
#             # Adaptive average pooling to 4x4x4
#             pooled = F.adaptive_avg_pool3d(state, (4, 4, 4))
#             pooled_states.append(pooled)
        
#         # Concatenate the pooled states
#         return torch.cat(pooled_states, dim=1)


In [16]:
#  Create an instance of the feature extractor
feature_extractor = FeatureExtractorSwinUNETR(
    img_size=(64, 64, 64),
    in_channels=in_channels,
    out_channels=out_channels,
    feature_size=48,
    use_checkpoint=True,
).to(device)
feature_extractor.load_state_dict(model.state_dict())
feature_extractor.eval()

FeatureExtractorSwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(1, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0-1): 2 x SwinTransformerBlock(
            (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=48, out_features=144, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=48, out_features=48, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=48, out_features=192, bias=True)
              (linear2): L

In [17]:

# # 3. Extract features from your dataset
# def extract_features_from_dataset(dataset, feature_extractor):
#     features = []
#     labels = []
    
#     for data in dataset:
#         inputs = torch.cat([data[key] for key in modality_keys], dim=1).to(device)
#         with torch.no_grad():
#             feature = feature_extractor.extract_features(inputs)
        
#         # Assuming the feature is 5D (batch, channels, depth, height, width)
#         # We'll flatten it to 2D (batch, features)
#         feature = feature.view(feature.size(0), -1)
        
#         features.append(feature.cpu().numpy())
#         labels.append(data['label'].cpu().numpy())  # Assuming you have labels in your dataset
    
#     return np.concatenate(features), np.concatenate(labels)


In [21]:
def extract_features_from_dataset(dataloader, feature_extractor):
    features = []
    labels = []
    
    for batch in dataloader:
        inputs = torch.cat([batch[key] for key in modality_keys], dim=1).to(device)
        
        with torch.no_grad():
            feature = feature_extractor.extract_features(inputs)
        
        features.append(feature.cpu().numpy())
        # labels.append(batch['label'].cpu().numpy())  # Uncomment if you have labels in your dataset
    
    final_features = np.concatenate(features)
    print(f"Shape of final extracted features: {final_features.shape}")
    
    return final_features

In [19]:
# def extract_features_from_dataset(dataloader, feature_extractor):
#     features = []
#     labels = []
    
#     for batch in dataloader:
#         inputs = torch.cat([batch[key] for key in modality_keys], dim=1).to(device)
#         with torch.no_grad():
#             feature = feature_extractor.extract_features(inputs)
        
#         # Global average pooling to reduce spatial dimensions
#         feature = torch.mean(feature, dim=[2, 3, 4])
        
#         features.append(feature.cpu().numpy())
#         # labels.append(batch['label'].cpu().numpy())  # Assuming you have labels in your dataset
    
#     return np.concatenate(features)


In [22]:

# Extract features for training and validation sets
train_features = extract_features_from_dataset(train_loader, feature_extractor)
val_features = extract_features_from_dataset(val_loader, feature_extractor)


/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(
/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(
/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(
/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/monai/transforms/utils.py:606: UserWarning: Num foregrounds 1123200, Num backgrounds 0, unable to generate class balanced samples, setting `pos_ratio` to 1.
  warnings.warn(


Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated features: torch.Size([2, 1488])
Shape of concatenated featur

In [ ]:
# Define Random Forest Regressor model
rf_model = RandomForestRegressor(random_state=42)


In [ ]:
# Define hyperparameter search space for regression
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [ ]:
# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(rf_model, param_distributions=param_dist, n_iter=25, cv=5, 
                                   scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)


In [ ]:
# Fit the model
random_search.fit(train_features, train_labels)


/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
35 fits failed out of a total of 125.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/m1/23CS60R48/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/base.py", line 666, in _validate_p

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   random_state=42, scoring='neg_mean_squared_error')

In [ ]:
# Get the best model
best_rf_model = random_search.best_estimator_
print("Best parameters found: ", random_search.best_params_)


Best parameters found:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 50}


In [ ]:
# Predict on validation set
val_pred = best_rf_model.predict(val_features)


In [ ]:
# Calculate regression metrics
mse = mean_squared_error(val_labels, val_pred)
rmse = np.sqrt(mse)
r2 = r2_score(val_labels, val_pred)
mae = mean_absolute_error(val_labels, val_pred)


In [ ]:
print(f"Validation Mean Squared Error: {mse}")
print(f"Validation Root Mean Squared Error: {rmse}")
print(f"Validation R-squared Score: {r2}")
print(f"Validation Mean Absolute Error: {mae}")

Validation Mean Squared Error: 165817.2415816085
Validation Root Mean Squared Error: 407.20663253636786
Validation R-squared Score: -0.003197829052890455
Validation Mean Absolute Error: 304.2128802130667


In [ ]:
# Print feature shapes for reference
print("Train features shape:", train_features.shape)
print("Validation features shape:", val_features.shape)

Train features shape: (188, 1488)
Validation features shape: (48, 1488)
